# Prepare data

In [82]:
!pip install kaggle-environments==1.14.14
!pip install evaluate
!pip install sentence-transformers==2.2.2
!pip install faiss-cpu
!pip install rouge_score
!pip install bert-score
!pip install trl==0.7.4
!pip install transformers==4.31.0
# !pip install langdetect

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 93.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 76.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0
  Using cached transformers-4.31.0-py3-none-any.whl.metadata (116 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing install

In [83]:
import trl
import transformers
import sentence_transformers
print("Transformers version:", transformers.__version__)
print("TRL version:", trl.__version__)
print("sentence_transformers version:", sentence_transformers.__version__)

Transformers version: 4.31.0
TRL version: 0.7.4
sentence_transformers version: 2.2.2


In [ ]:
import asyncio
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
import torch
import os
import faiss
import pandas as pd
import evaluate
import random
import shutil
from sentence_transformers import SentenceTransformer
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from torch.amp import autocast, GradScaler
import re
# from langdetect import detect
from tqdm import tqdm
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
    DataCollatorForSeq2Seq,
    GenerationConfig,
    EarlyStoppingCallback,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
from trl import (
    PPOConfig, 
    PPOTrainer,
    AutoModelForSeq2SeqLMWithValueHead,
    create_reference_model
)

# Cấu hình thiết bị và biến môi trường cho multi-GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  # Sử dụng 2 GPU đầu tiên
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Đọc file xlsx
file_path = "/kaggle/input/reward-data/reward_res_sheet.xlsx"
df = pd.read_excel(file_path, engine="openpyxl")
# Chuyển cột clause thành chuỗi
df['article'] = df['article'].astype(str)
df['clause'] = df['clause'].astype(str)
df['point'] = df['point'].astype(str)
# Đổi tên cột thủ công
df.rename(columns={
    'Legal Acc (pháp lý)': 'Legal_Acc',
    'Relevance (ý trọng tâm)': 'Relevance',
    'Fluency (Ngữ pháp)': 'Fluency'
}, inplace=True)

# Định nghĩa quy tắc chuyển đổi
mapping = {1: 0.0, 2: 0.5, 3: 1.0}
# Áp dụng cho các cột cần thiết
df[['Legal_Acc', 'Relevance', 'Fluency']] = df[['Legal_Acc', 'Relevance', 'Fluency']].apply(lambda x: x.map(mapping))
# df_data = pd.concat([df.iloc[:400], df.iloc[2215:]], ignore_index=True)

# Đọc file CSV vào Dataset
dataset = Dataset.from_pandas(df)

# Hiển thị 5 dòng đầu tiên
df.head()

,conversation_id,article,clause,point,question,target_text,generated_text,Legal_Acc,Relevance,Fluency,bleu_score,rouge1_score,rouge2_score,rougeL_score,rougeLsum_score
0,8,120,2,nan,Nếu một bên cố ý cản trở điều kiện phát sinh h...,"Theo khoản 2, điều 120, Cản trở thì coi như xả...","Theo Khoản 2, Điều 120, Nếu cố ý cản trở thì c...",1.0,1.0,0.5,0.279272,0.794118,0.666667,0.794118,0.794118
1,19,686,nan,nan,Nếu tự ý làm việc giúp người khác ở nước ngoài...,"Theo điều 686, Các bên liên quan có thể thỏa t...","Theo Điều 687 Bộ luật Dân sự 2015, các bên đượ...",0.5,1.0,1.0,0.278530,0.824859,0.617143,0.700565,0.700565
2,200,63,2,nan,Khoản 2 Điều 63 Bộ luật Dân sự quy định như th...,"Theo khoản 2, điều 63, quy định trường hợp ngư...","Theo Khoản 2, Điều 63, quy định trường hợp ngư...",1.0,1.0,1.0,0.499364,0.797428,0.724919,0.713826,0.713826
3,95,34,3,nan,Khoản 3 Điều 34 Bộ luật Dân sự quy định như th...,"Theo khoản 3, điều 34, quy định thông tin sai ...","Theo Khoản 3, Điều 34, quy định thông tin ảnh ...",1.0,1.0,1.0,0.154495,0.447489,0.387097,0.374429,0.374429
4,184,515,2,nan,Bên sử dụng dịch vụ có phải trả tiền dịch vụ k...,"Theo khoản 2, điều 515, Bên sử dụng dịch vụ có...","Theo khoản 3 Điều 519 Bộ luật Dân sự 2015, bên...",0.0,1.0,1.0,0.369260,0.740741,0.582278,0.716049,0.716049


In [85]:
dataset

Dataset({
    features: ['conversation_id', 'article', 'clause', 'point', 'question', 'target_text', 'generated_text', 'Legal_Acc', 'Relevance', 'Fluency', 'bleu_score', 'rouge1_score', 'rouge2_score', 'rougeL_score', 'rougeLsum_score'],
    num_rows: 624
})

In [86]:
dataset[123]

{'conversation_id': 24,
 'article': '440',
 'clause': '1',
 'point': 'nan',
 'question': 'Nghĩa vụ trả tiền của bên mua trong hợp đồng mua bán là gì?',
 'target_text': 'Theo khoản 1, điều 440, Bên mua có trách nhiệm thanh toán tiền đúng thời hạn, địa điểm và số tiền đã thỏa thuận trong hợp đồng.',
 'generated_text': 'Theo khoản 1 Điều 434 Bộ luật Dân sự 2015, bên mua thanh toán tiền theo thời gian thỏa thuận, nếu không xác định hoặc xác định không rõ ràng thì phải thanh toán ngay khi nhận tài sản hoặc giấy tờ chứng nhận quyền sở hữu.',
 'Legal_Acc': 0.0,
 'Relevance': 1.0,
 'Fluency': 1.0,
 'bleu_score': 0.0,
 'rouge1_score': 0.5043478260869565,
 'rouge2_score': 0.2831858407079646,
 'rougeL_score': 0.3999999999999999,
 'rougeLsum_score': 0.3999999999999999}

# Reward Model

In [109]:
print(dataset)
# In kiểu dữ liệu của từng cột trong dataset
print(dataset.features)

Dataset({
    features: ['conversation_id', 'article', 'clause', 'point', 'question', 'target_text', 'generated_text', 'Legal_Acc', 'Relevance', 'Fluency', 'bleu_score', 'rouge1_score', 'rouge2_score', 'rougeL_score', 'rougeLsum_score'],
    num_rows: 624
})
{'conversation_id': Value(dtype='int64', id=None), 'article': Value(dtype='string', id=None), 'clause': Value(dtype='string', id=None), 'point': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'target_text': Value(dtype='string', id=None), 'generated_text': Value(dtype='string', id=None), 'Legal_Acc': Value(dtype='float64', id=None), 'Relevance': Value(dtype='float64', id=None), 'Fluency': Value(dtype='float64', id=None), 'bleu_score': Value(dtype='float64', id=None), 'rouge1_score': Value(dtype='float64', id=None), 'rouge2_score': Value(dtype='float64', id=None), 'rougeL_score': Value(dtype='float64', id=None), 'rougeLsum_score': Value(dtype='float64', id=None)}


In [106]:
dataset[0]

{'conversation_id': 8,
 'article': '120',
 'clause': '2',
 'point': 'nan',
 'question': 'Nếu một bên cố ý cản trở điều kiện phát sinh hoặc hủy bỏ giao dịch dân sự thì sao?',
 'target_text': 'Theo khoản 2, điều 120, Cản trở thì coi như xảy ra, thúc đẩy thì coi như không xảy ra.',
 'generated_text': 'Theo Khoản 2, Điều 120, Nếu cố ý cản trở thì coi như điều kiện đã xảy ra; nếu cố ý thúc đẩy thì coi như điều kiện không xảy ra.',
 'Legal_Acc': 1.0,
 'Relevance': 1.0,
 'Fluency': 0.5,
 'bleu_score': 0.2792720350291165,
 'rouge1_score': 0.7941176470588235,
 'rouge2_score': 0.6666666666666667,
 'rougeL_score': 0.7941176470588235,
 'rougeLsum_score': 0.7941176470588235}

In [148]:
import shutil
import os

folder_path = "/kaggle/working/reward_model"

# Xóa thư mục nếu tồn tại
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Đã xóa thư mục: {folder_path}")
else:
    print(f"Thư mục không tồn tại: {folder_path}")

Đã xóa thư mục: /kaggle/working/reward_model


In [149]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

# Sử dụng PhoBERT (hoặc mô hình BERT cho tiếng Việt) để làm Reward Model
reward_model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=1).to(device)

# Tải tokenizer cho PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# # Chuyển đổi Dataset thành DataFrame (nếu chưa làm)
# reward_df = reward_dataset.to_pandas()

# # Chuyển lại DataFrame thành Dataset sau khi xử lý
# reward_dataset = Dataset.from_pandas(reward_df)

# Định nghĩa hàm tiền xử lý
def preprocess_reward_data(examples):
    # Lấy cột generated_text từ examples
    texts = examples["generated_text"]
    # Tokenize văn bản
    inputs = tokenizer(texts, max_length=256, truncation=True, padding="max_length")

    # Chuyển các cột thành float trước khi tính toán
    legal_acc = [float(x) for x in examples["Legal_Acc"]]
    relevance = [float(x) for x in examples["Relevance"]]
    fluency = [float(x) for x in examples["Fluency"]]
    bleu_score = [float(x) for x in examples["bleu_score"]]
    rouge1_score = [float(x) for x in examples["rouge1_score"]]

    # Tính toán nhãn
    # inputs["labels"] = [
    #     (0.2 * la + 0.2 * r + 0.2 * f + 0.2 * b + 0.2 * rg)*2-1
    #     for la, r, f, b, rg in zip(legal_acc, relevance, fluency, bleu_score, rouge1_score)
    # ]
    inputs["labels"] = [
            (0.3 * la + 0.3 * r + 0.3 * f + 0.05 * b + 0.05 * rg)
            for la, r, f, b, rg in zip(legal_acc, relevance, fluency, bleu_score, rouge1_score)
        ]

    return inputs

# Áp dụng hàm tiền xử lý lên dataset và chuyển đổi thành dataset đã tokenized
tokenized_reward_data = dataset.map(preprocess_reward_data, batched=True)

# Chia dataset thành train và eval (10% cho eval)
train_test_ds = tokenized_reward_data.train_test_split(test_size=0.2)
train_ds = train_test_ds["train"]
eval_ds = train_test_ds["test"]

# Huấn luyện Reward Model với Early Stopping
reward_args = TrainingArguments(
    output_dir="./reward_model",
    evaluation_strategy="epoch",        # Đánh giá sau mỗi epoch
    save_strategy="epoch",              # Lưu mô hình sau mỗi epoch
    logging_dir="./logs_reward",
    per_device_train_batch_size=8,
    num_train_epochs=10,
    fp16=True,
    load_best_model_at_end=True,         # Load mô hình tốt nhất khi kết thúc
    metric_for_best_model="eval_loss",   # Theo dõi metric eval_loss
    greater_is_better=False              # eval_loss càng thấp càng tốt
)

reward_trainer = Trainer(
    model=reward_model,
    args=reward_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

reward_trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/624 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.065238
2,No log,0.079179
3,No log,0.108364
4,No log,0.077006
5,No log,0.088163
6,No log,0.083755


/opt/conda/lib/python3.10/site-packages/transformers/trainer.py:2159: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(best_model_path, map_location="cp

TrainOutput(global_step=378, training_loss=0.05328344415735315, metrics={'train_runtime': 107.9452, 'train_samples_per_second': 46.227, 'train_steps_per_second': 5.836, 'total_flos': 393873713409024.0, 'train_loss': 0.05328344415735315, 'epoch': 6.0})

In [150]:
print(f"Best model saved at: {reward_args.output_dir}/checkpoint-{reward_trainer.state.best_model_checkpoint}")

Best model saved at: ./reward_model/checkpoint-./reward_model/checkpoint-63


In [151]:
reward_trainer.state.best_model_checkpoint

'./reward_model/checkpoint-63'

In [152]:
path_best_model=reward_trainer.state.best_model_checkpoint.replace('.', '/kaggle/working', 1)
reward_model = AutoModelForSequenceClassification.from_pretrained(path_best_model).to(device)

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:460: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

In [153]:
# Lưu mô hình và tokenizer
reward_model.save_pretrained("./reward_model_1")
tokenizer.save_pretrained("./reward_model_1")

('./reward_model_1/tokenizer_config.json',
 './reward_model_1/special_tokens_map.json',
 './reward_model_1/vocab.txt',
 './reward_model_1/bpe.codes',
 './reward_model_1/added_tokens.json')

In [158]:
def generate_reward_score(text, model_path="./reward_model_1", device="cpu"):
    """Generate a reward score for a given text using the trained reward model."""
    # Load model and tokenizer
    model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
    tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

    # Tokenize input text
    inputs = tokenizer(text, max_length=256, truncation=True, padding="max_length", return_tensors="pt").to(device)

    # Make prediction
    with torch.no_grad():
        output = model(**inputs)
        score = output.logits.item()

    return score
sample_text = "Theo Khoản 1, Điều 162, Chủ sở hữu không chịu trách nhiệm về rủi ro đối với tài sản; chỉ chịu trách nhiệm rủi ro đối với tài sản trước khi bị hủy."
reward = generate_reward_score(sample_text)
print(f"Reward Score: {reward}")
sample_text = "Theo Khoản 2, Điều 656, Mọi thỏa thuận của những người thừa kế phải được lập thành văn bản."
reward = generate_reward_score(sample_text)
print(f"Reward Score: {reward}")
sample_text = "Theo Điều 576, quy định người cócôngviệc được NTKVKVKVKVKVKVKV thực hiện phải tiếp nhận công việc và thanh toán các chi phí hợp lý mà người có côngreau bỏ ra để thực hiện công việc; trả thù lao, cấp dưỡng, Tcherniastan, Mnodia và xung quanh."
reward = generate_reward_score(sample_text)
print(f"Reward Score: {reward}")
sample_text = "Theo Điều 12, Nội dung chính của Điều 12 ga BRFKYVTIST: PotamogetonNTKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKV"
reward = generate_reward_score(sample_text)
print(f"Reward Score: {reward}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Reward Score: 0.6669183969497681


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Reward Score: 0.7199738025665283


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Reward Score: 0.25690117478370667


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Reward Score: 0.09409703314304352


In [155]:
dataset[600]

{'conversation_id': 8,
 'article': 'N\\A',
 'clause': 'N\\A',
 'point': 'N\\A',
 'question': 'Khi tự bảo vệ quyền dân sự, cần phải tuân thủ những nguyên tắc nào?',
 'target_text': 'Theo điều 12, Tự bảo vệ quyền dân sự một cách phù hợp nghĩa là các biện pháp tự vệ mà chủ thể áp dụng phải tương xứng với tính chất, mức độ của hành vi xâm phạm đến quyền dân sự của mình, đồng thời không được vi phạm các nguyên tắc cơ bản của pháp luật dân sự như nguyên tắc thiện chí, trung thực, không xâm phạm đến lợi ích công cộng, v.v..',
 'generated_text': 'Theo Điều 12, Nội dung chính của Điều 12 ga BRFKYVTIST: PotamogetonNTKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKVKV

In [157]:
# Nén thư mục thành file ZIP
shutil.make_archive("Reward_ViT5_QAChatbot", "zip", "./reward_model_1")

from IPython.display import FileLink

# Tạo liên kết tải xuống
file_link = FileLink(r'Reward_ViT5_QAChatbot.zip')
display(file_link)

print("Model and tokenizer saved successfully!")

/kaggle/working/Reward_ViT5_QAChatbot.zip

Model and tokenizer saved successfully!
